In [21]:
import os
import requests
from pathlib import Path
import zipfile
import pandas as pd
from pathlib import Path
import duckdb

### Extracción de datos

In [22]:

# Endpoint del dataset
url = "https://api.biodiversitydata.nl/v2/specimen/dwca/getDataSet/wildlife-sounds-birds"

# Carpeta Downloads del usuario
downloads_folder = Path.home() / "Downloads"
downloads_folder.mkdir(exist_ok=True)

# Nombre del ZIP y carpeta de extracción
zip_path = downloads_folder / "wildlife-sounds-birds_dwca.zip"
extract_folder = downloads_folder / "wildlife-sounds-birds_dwca"

print(f"➡ El ZIP se guardará en: {zip_path}")
print(f"➡ Los archivos se extraerán en: {extract_folder}")

# Cabeceras opcionales
headers = {
    "User-Agent": "AntonioRomero-data-science-project/1.0"
}

# DESCARGA
response = requests.get(url, headers=headers, stream=True)
print("Status code:", response.status_code)

if response.status_code == 200:
    with open(zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f" Descarga completada: {zip_path}")
else:
    print("Error al descargar el dataset")
    print("Respuesta del servidor (primeros 500 chars):")
    print(response.text[:500])
    raise SystemExit()

# EXTRACCIÓN
# Crear carpeta de extracción si no existe
extract_folder.mkdir(exist_ok=True)

# Descomprimir ZIP
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Extracción completada en: {extract_folder}")

# LISTAR ARCHIVOS
print("\n Contenido extraído:")
for root, dirs, files in os.walk(extract_folder):
    level = root.replace(str(extract_folder), "").count(os.sep)
    indent = "  " * level
    print(f"{indent}- {os.path.basename(root)}/")
    subindent = "  " * (level + 1)
    for f in files:
        print(f"{subindent}• {f}")



➡ El ZIP se guardará en: C:\Users\anto-\Downloads\wildlife-sounds-birds_dwca.zip
➡ Los archivos se extraerán en: C:\Users\anto-\Downloads\wildlife-sounds-birds_dwca
Status code: 200
 Descarga completada: C:\Users\anto-\Downloads\wildlife-sounds-birds_dwca.zip
Extracción completada en: C:\Users\anto-\Downloads\wildlife-sounds-birds_dwca

 Contenido extraído:
- wildlife-sounds-birds_dwca/
  • eml.xml
  • meta.xml
  • Multimedia.txt
  • Occurrence.txt


In [23]:
# Leer las primeras lineas de Occurrence.txt
folder = Path.home() / "Downloads" / "wildlife-sounds-birds_dwca"
occurrence_file = folder / "Occurrence.txt"

with open(occurrence_file, "r", encoding="utf-8") as f:
    for i in range(10):
        print(f.readline().rstrip())


id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,scientificName,taxonRank,kingdom,family,higherClassification,nomenclaturalCode,identificationRemarks,vernacularName,longitudeDecimal,latitudeDecimal,geodeticDatum,country,locality,verbatimElevation,recordedBy,eventDate,eventTime,verbatimEventDate,fieldNotes,behavior,sex,lifeStage,preparations,references,Associated Taxa,rightsHolder,license
9@XC,https://data.biodiversitydata.nl/xeno-canto/observation/XC9,XC9,HumanObservation,Wildlife sounds - Birds,,,Synallaxis,azarae,media,Synallaxis azarae media,subspecies,Animalia,Furnariidae,Animalia|PASSERIFORMES|Furnariidae,ICZN,,Azara's Spinetail,-78.7,-2.6,WGS84,Ecuador,"La Libertad, Cañar",3000 m,Bob Planqué,2002-08-12,07:00,12-08-2002,two birds trip:http://www.cs.bris.ac.uk/home/planque/Mazar/mazar-bird-report.html,song,,,field recording,https://data.biodiversitydata.nl/xeno-canto/observation/XC9,,Bob Plan

In [24]:
folder = Path.home() / "Downloads" / "wildlife-sounds-birds_dwca"
occurrence_file = folder / "Occurrence.txt"

occ = pd.read_csv(
    occurrence_file,
    sep=None,        # autodetecta el separador
    engine="python"  # más flexible, acepta líneas irregulares
)

occ.head()


,﻿id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,...,verbatimEventDate,fieldNotes,behavior,sex,lifeStage,preparations,references,Associated Taxa,rightsHolder,license
0,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC9,HumanObservation,Wildlife sounds - Birds,NaN,NaN,Synallaxis,azarae,media,...,12-08-2002,two birds trip:http://www.cs.bris.ac.uk/home/p...,song,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Bob Planqué,CC BY-NC
1,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC9,HumanObservation,Wildlife sounds - Birds,NaN,NaN,Synallaxis,azarae,media,...,12-08-2002,two birds trip:http://www.cs.bris.ac.uk/home/p...,song,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Bob Planqué,CC BY-NC
2,99@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC99,HumanObservation,Wildlife sounds - Birds,NaN,NaN,Turdus,hauxwelli,NaN,...,01-10-2003,trip:http://www.cs.bris.ac.uk/home/planque/Peru/,song,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Bob Planqué,CC BY-NC
3,99@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC99,HumanObservation,Wildlife sounds - Birds,NaN,NaN,Turdus,hauxwelli,NaN,...,01-10-2003,trip:http://www.cs.bris.ac.uk/home/planque/Peru/,song,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,Bob Planqué,CC BY-NC
4,9999@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC9999,HumanObservation,Wildlife sounds - Birds,NaN,NaN,Myrmothera,campanisona,signata,...,04-03-1998,NaN,song,NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Selenidera reinwardtii,Allen T. Chartier,CC BY-NC


In [25]:
occ.info

<bound method DataFrame.info of                 ﻿id                                       occurrenceID  \
0              9@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
1              9@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
2             99@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
3             99@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
4           9999@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
...             ...                                                ...   
1638815  1000004@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
1638816  1000003@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
1638817  1000002@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
1638818  1000001@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   
1638819  1000000@XC  https://data.biodiversitydata.nl/xeno-canto/ob...   

        catalogNumber       basisOfRecord           collectionCode  \
0        

### Acotamos para España

In [26]:
# Rutas principales de los archivos
folder = Path.home() / "Downloads" / "wildlife-sounds-birds_dwca"
occ_path = (folder / "Occurrence.txt").as_posix()
mult_path = (folder / "Multimedia.txt").as_posix()
out_csv = (Path.home() / "Downloads" / "birds_spain_joined.csv").as_posix()

print("📂 Occurrence:", occ_path)
print("📂 Multimedia:", mult_path)
print("📂 Output CSV:", out_csv)

# Conexión DuckDB en memoria
con = duckdb.connect(database=":memory:")

# Crear tabla solo España
con.execute(f"""
    CREATE OR REPLACE TABLE occurrence_spain AS
    SELECT *
    FROM read_csv_auto('{occ_path}')
    WHERE country = 'Spain';
""")

# Crear tabla multimedia
con.execute(f"""
    CREATE OR REPLACE TABLE multimedia AS
    SELECT *
    FROM read_csv_auto('{mult_path}');
""")

# Ver columnas (control rápido)
print("\nColumnas occurrence_spain:")
print(con.execute("PRAGMA table_info(occurrence_spain);").fetchdf().head())

print("\nColumnas multimedia:")
print(con.execute("PRAGMA table_info(multimedia);").fetchdf().head())

# JOIN por id (Occurrence) y coreid (Multimedia)
con.execute("""
    CREATE OR REPLACE TABLE birds_spain AS
    SELECT
        o.*,
        m.identifier AS audio_url
    FROM occurrence_spain o
    LEFT JOIN multimedia m
        ON o.id = m.coreid;
""")

📂 Occurrence: C:/Users/anto-/Downloads/wildlife-sounds-birds_dwca/Occurrence.txt
📂 Multimedia: C:/Users/anto-/Downloads/wildlife-sounds-birds_dwca/Multimedia.txt
📂 Output CSV: C:/Users/anto-/Downloads/birds_spain_joined.csv


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Columnas occurrence_spain:
   cid            name     type  notnull dflt_value     pk
0    0              id  VARCHAR    False       None  False
1    1    occurrenceID  VARCHAR    False       None  False
2    2   catalogNumber  VARCHAR    False       None  False
3    3   basisOfRecord  VARCHAR    False       None  False
4    4  collectionCode  VARCHAR    False       None  False

Columnas multimedia:
   cid                            name     type  notnull dflt_value     pk
0    0                          CoreId  VARCHAR    False       None  False
1    1  associatedObservationReference  VARCHAR    False       None  False
2    2                      Identifier  VARCHAR    False       None  False
3    3                            type  VARCHAR    False       None  False
4    4                          Rating   BIGINT    False       None  False


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [27]:
# Exportar a CSV
con.execute(f"""
    COPY birds_spain
    TO '{out_csv}'
    WITH (HEADER, DELIMITER ',');
""")

print("CSV creado en:", out_csv)

# Cargar en pandas
df = pd.read_csv(out_csv)
df.head()

CSV creado en: C:/Users/anto-/Downloads/birds_spain_joined.csv


C:\Users\anto-\AppData\Local\Temp\ipykernel_34256\2700074677.py:11: DtypeWarning: Columns (5,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(out_csv)


,id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,...,fieldNotes,behavior,sex,lifeStage,preparations,references,Associated Taxa,rightsHolder,license,audio_url
0,818772@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC818772,MachineObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Marantz PMD-661 MKII"",""mic...",NaN,Lanius,senator,NaN,...,animal seen:no; playback used:no,"flight call, nocturnal flight call",NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,José Manuel Reyes Páez,CC BY-NC,https://xeno-canto.org/sounds/uploaded/UJJDAEG...
1,818772@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC818772,MachineObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Marantz PMD-661 MKII"",""mic...",NaN,Lanius,senator,NaN,...,animal seen:no; playback used:no,"flight call, nocturnal flight call",NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,José Manuel Reyes Páez,CC BY-NC,https://xeno-canto.org/sounds/uploaded/UJJDAEG...
2,818772@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC818772,MachineObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Marantz PMD-661 MKII"",""mic...",NaN,Lanius,senator,NaN,...,animal seen:no; playback used:no,"flight call, nocturnal flight call",NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,José Manuel Reyes Páez,CC BY-NC,https://xeno-canto.org/sounds/uploaded/UJJDAEG...
3,818772@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC818772,MachineObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Marantz PMD-661 MKII"",""mic...",NaN,Lanius,senator,NaN,...,animal seen:no; playback used:no,"flight call, nocturnal flight call",NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,José Manuel Reyes Páez,CC BY-NC,https://xeno-canto.org/sounds/uploaded/UJJDAEG...
4,818772@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC818772,MachineObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Marantz PMD-661 MKII"",""mic...",NaN,Lanius,senator,NaN,...,animal seen:no; playback used:no,"flight call, nocturnal flight call",NaN,NaN,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,NaN,José Manuel Reyes Páez,CC BY-NC,https://xeno-canto.org/sounds/uploaded/UJJDAEG...


In [28]:
con.execute("PRAGMA table_info(occurrence_spain);").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,id,VARCHAR,False,None,False
1,1,occurrenceID,VARCHAR,False,None,False
2,2,catalogNumber,VARCHAR,False,None,False
3,3,basisOfRecord,VARCHAR,False,None,False
4,4,collectionCode,VARCHAR,False,None,False
5,5,dynamicProperties,VARCHAR,False,None,False
6,6,otherCatalogNumbers,VARCHAR,False,None,False
7,7,genus,VARCHAR,False,None,False
8,8,specificEpithet,VARCHAR,False,None,False
9,9,infraspecificEpithet,VARCHAR,False,None,False
